In [ ]:
knitr::opts_chunk$set(echo = TRUE, eval = TRUE)



## Intro 

In this section we will be working with shapefiles. More specifically how to read in a shapefile and join this to our aggregated crime count data frame. From there we introduce classification methods as a way to better visualize crime counts. 

## Load packages

As always the first step is to load the necessary R packages via the library function. If you do not have these packages installed then please follow the instructions in the *Preliminary Task.Rmd* file. 


In [ ]:
# for data reading/manipulation 
library(dplyr)
library(tidyr)
library(readr)
library(tibble)
library(janitor)
# for spatial data and gis
library(sf)
library(ggmap)
library(ggplot2)
library(ggspatial)
library(spdep)
library(leaflet) 
library(RColorBrewer)
library(tmap)


## What is a Shapefile? 

They represent a geospatial vector that is used for GIS software. Shapefiles store both geogrpahic location and its associated attribute information 

The Shapefile format stores the data as primitive geometric shapes like points, lines, and polygons. These shapes, together with data attributes that are linked to each shape, create the representation of the geographic data.

They contain four mandatory file extensions (.shx, .shp, .dbf and the .prj). 
- The .shp contains the geometry data (a 2D axis ordering of coordinate data)
- The .shx contains the positional index of the feature geometry 
- The .dbf contins the attributes for each shape
- The .prj contains the cs and projection information

In criminological research, the LSOA is quite frequently used as the main census geography 


## Where to obtain shapefiles 

I collected my shapefile data via the UKDS Census Support *(https://borders.ukdataservice.ac.uk/bds.html)*. If you want specific information about how to use the website, please refer to the *Downloading the data* document again 


## Read in the Shapefile for 'Surrey Heath' 


In [ ]:
shp_file <- sf::st_read("~/Documents/Crime_R_Workshop/Crime_Data_in_R/March_2023/Data/Shapefile/england_lsoa_2021.shp")



You can also use the head() function in shapefiles!



In [ ]:
head(shp_file)
#or use 'View(shp_file) to view the full dataset which will open up in a new panel


*Further Information - To clarify, this is an 'empty shapefile', it simply contains the boundary profile of Surrey Heath and does not contain any further attribute information. However, if it did contain further attribute information such as the crime counts, population statistics, IMD counts, then you would not need to join the data as we do in this workshop, but instead you could layer the shapefile over our simple feature object created in Topic 1. More information on this method is available from the workshop that was held in February - all resources are available via the 'Feb_2021' folder from the github link [https://github.com/UKDataServiceOpen/Crime_Data_in_R.git] *

Lets plot the empty shapefile for Surrey Heath to see what we're actually looking at.


In [ ]:
## Plot the Shapefile 
ggplot() + 
  geom_sf(data = shp_file)


As you can seem, we have a map that details the borders (i.e. shape) between each LSOA in Surrey. 

This workshop instead joins the 'crime' datatset (in tibble format) to the above shapefile. Our newly created object 'shp_file' is in fact a sf object, which is short for a 'simple feature object'. You can check this by typing *class(shp_file)*. 


In [ ]:
class(shp_file)



So in total, the shapefile consits of 5 variables. The first 4 variables indicate information about that specific LSOA, we are given the name, LSOA code and LSOA name. We can ignore the column 'label' as this is just another reference point.


The column I want to draw attention to is the 'geometry column' 


In [ ]:
attributes(shp_file$geometry)



The geometry column can be split into two key sections; the feature and the geometry 
  - The feature in this case is our polygon level (referenced by the multipolygon) which is in fact a  *simple feature geometery list- column (sfc)* 
  - The geometery are the numbers that follow, and more technically known as a *'simple feature geometry (sfg)*

The column in the sf data.frame that contains the geometries is a list, of class sfc. We can retrieve the geometry list-column in this case by using st_geometry. 


In [ ]:
st_geometry(shp_file)



Now we have a basic understanding of what a shapefile is and how we can import them into r, the next step is run some data manipulation and create some new dataframes that can work with the format of shapefiles. 



## Group the crimes per lsoa 

The original crime data set contains the individual count of reported crime types across LSOAS, therefore the LSOAs are repeated multiple times. This is because you would expect to see multiple crime counts in one LSOA.

In order to highlight how many crimes have occurred in each LSOA, we can count the crimes per LSOA and obtained grouped statistics.


In [ ]:
crimes_grouped_by_lsoa <- crime %>%
  group_by(lsoa_code) %>%
  summarise(count=n())

head(crimes_grouped_by_lsoa)


## Merge the shapefile to the crime dataset

In our new object you will see two variables, the LSOA and the count of crime in each one.  

We can now join the Shapefile (the geospatial vector) and the crimes_grouped_by_losa (the aggregated data)

To join the crimes per lsoa to the shapefile we can use the left_join function that returns all the rows of the table on the left side of the join and matching rows for the table on the right side of join.


In [ ]:
surrey_lsoa <- left_join(shp_file, crimes_grouped_by_lsoa, by = c("lsoa21cd" = "lsoa_code"))

head(surrey_lsoa)

st_geometry_type(surrey_lsoa)    #view the geometery type 
st_bbox(surrey_lsoa)             #obtains the objects value as specific units 


#The spatial extent of a shapefile or R spatial object represents the geographic “edge” or location that is the #furthest north, south east and west. Thus is represents the overall geographic coverage of the spatial object.


Now lets map the new data!



In [ ]:
#map the data
ggplot() + 
  annotation_map_tile() + 
  geom_sf(data = surrey_lsoa, aes(fill = count), alpha = 0.5) + 
  scale_fill_gradient2(name ="Number of crimes")


## Plotting via the 'tmap' package

The tmap package allows you tp create thematic maps, the syntax is very similar to the ggplot2. Each map can be plot as an image or as an interactive map via the *tmap_mode("view" / "plot" )* function. 


In [ ]:
tmap_mode("view")

tm_shape(surrey_lsoa) + 
  tm_fill("count") + 
  tm_borders("green", lwd = 0.7, alpha = 0.5)
  #tm_text("name", size = "AREA", col = "black")
  #tmap_style("col_blind")


### Classification methods

How can we better visualise counts? Count data does not equally represent the population distribution at hand, tmaps allows you to alter the characteristics of theamatic maps via the 'styles' function. The different styles result in different binning techniques.  Now, when mapping quantitiatve data such as crime counts, typicaly the variables needed to be put into to 'bins'. As seen in the previous example, the default binning applied to highlight the LSOAs grouped started from 1-10, 11-20, 21-20, 31-40, 41-50 and 51-60 crimes.

These bins were decided on automatically, however we can define more accurate classes that best refelct the distributional character of the data set.


In this example I've used the "kmeans", "jenks" and "sd".

- k-means is a method of vector quantisation, originally from signal processing, that aims to partition n observations into k clusters in which each observation belongs to the cluster with the nearest mean, serving as a prototype of the cluster.
- Jenks (also known as natural breaks, or goodness of fit variance) classifcaiton aims to arrange a set of values into natural classes, that is the most optimal class range found naturally. This method minimizes the variation within each range, so the areas within each range are as close as possible in value to each other. 
- sd classification is a standarised measure of observations deviated from the mean. By showing which values are above or below the mean, this method helps to show which locations are above or below an average mean.


In [ ]:
a <- tm_shape(surrey_lsoa) + 
  tm_fill("count", style = "kmeans") + 
  tm_borders(alpha = 0.3)

b <- tm_shape(surrey_lsoa) + 
  tm_fill("count", style = "jenks") + 
  tm_borders(alpha = 0.3)

c <- tm_shape(surrey_lsoa) + 
  tm_fill("count", style = "sd") + 
  tm_borders(alpha = 0.3)


## tmap_arrange

tmap_mode("plot")
tmap_arrange(a, b, c)


## Using categorical variables (tm_facets)

Just like the tmap_arrange function, tmap_facets are way to produce side-by-side maps (known as *small multiples*). It is similar to the 'facet_grid' function in ggplot2

Following the rdocumentation [https://www.rdocumentation.org/packages/tmap/versions/3.3-2/topics/tm_facets] "Small multiples can be created in two ways: 1) by specifying the by argument with one or two variable names, by which the data is grouped, 2) by specifying multiple variable names in any of the aesthetic argument of the layer functions (for instance, the argument col in tm_fill)." 

Typically tm_facets are defined by a categorical variables. For example in this example, I am using tm_facets() to seperate the map into multiple components by lsoa (This isnt't the best example of a categorical variable, but something like the urban or rural landscape, or deprivation decile, would be more of interest in criminology). You could for example download the 2011 rural/urban classification from open geography portal and join this to our 'surrey_lsoa' sf object (using left_join). 

If you are interested in doing so the dataset can be found here;
[https://www.ons.gov.uk/methodology/geography/geographicalproducts/ruralurbanclassifications/2011ruralurbanclassification]


In [ ]:
tm_shape(surrey_lsoa) +
  tm_fill("count",
          style = "quantile",
          palette = "Blues",
          thres.poly = 0) + 
  tm_facets(by="name", 
            free.coords=TRUE, 
            drop.shapes=TRUE) +
  tm_layout(legend.show = FALSE,
            title.position = c("center", "center"), 
            title.size = 20) +
  tm_borders(alpha = 0.5)


## Map Layouts - additional features of tmap (optional task)

### map style 


In [ ]:
tm_shape(surrey_lsoa) + 
  tm_fill("count", style = "sd") + 
  tm_borders(alpha = 0.3) + 
  tmap_style("col_blind")


### map legends



In [ ]:
tm_shape(surrey_lsoa)+
  tm_fill("count", 
          style = "quantile", 
          palette = "Blues", 
          legend.hist = TRUE, 
          legend.is.portrait = TRUE,
          legend.hist.z = 0.1) +
  tm_layout(legend.height = 0.45, 
            legend.width = 0.35,
            legend.outside = FALSE,
            legend.position = c("right", "bottom"),
            frame = FALSE) +
  tm_borders(alpha = 0.5)


### compass, scale bar and grid



In [ ]:
tm_shape(surrey_lsoa)+
  tm_fill("count", 
          style = "quantile", 
          palette = "Blues", 
          legend.hist = TRUE, 
          legend.is.portrait = TRUE,
          legend.hist.z = 0.1) +
  tm_layout(legend.height = 0.45, 
            legend.width = 0.35,
            legend.outside = FALSE,
            legend.position = c("right", "bottom"),
            frame = FALSE) +
  tm_borders(alpha = 0.5) +
  tm_compass(type="8star", size = 2) +   #compass 
  tm_scale_bar(width = 0.15) +           #scale bar 
  tm_grid()                              #grid


## Activity 2

1. Explore some of the different classification methods such as "bclust" and "hclust"  - what are the main differences? To get help on the different methods available use *??tmap-package* or search in the help tab 

2. Assign your new bclust and hclust classification maps into separate objects (call them "h" and "b" and plot them together using tmap_arrange()


3. Plot an interactive map using the "bclust" classification method by changing the command in the tmap_mode() function 

   


In [ ]:
#1)

... <- tm_shape(.....) + 
  tm_fill("...", style = "....") + 
  tm_borders(alpha = 0.3)


... <- tm_shape(........) + 
  tm_fill("...", style = ".....") + 
  tm_borders(alpha = 0.3)


#2) 

tmap_arrange(...., ....)



#3) 

tmap_mode("....")

tm_shape(.....) + 
  tm_fill(....., style = .....) + 
  tm_borders(alpha = 0.3)
